<a href="https://colab.research.google.com/github/aslikorkmaz48/NLP-Portfolio-T-rk-e/blob/main/02_Ozetleme_Projesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
from gensim.models import Word2Vec
from zeyrek import MorphAnalyzer
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity

# 1. TEMEL VERİ HAVUZU
temel_urunler = [
    "koşu ayakkabı pabuç spor konfor",
    "dağ bot yürüyüş outdoor su geçirmez",
    "kahve makine espresso çekirdek içecek",
    "akıllı saat takip nabız teknoloji",
    "kulaklık müzik ses gürültü engelleyici",
    "şarj kablo hızlı dolum enerji batarya",
    "televizyon ekran görüntü sinema panel",
    "bilgisayar dizüstü laptop performans oyuncu"
]

# 2. VERİ ARTIRMA (DATA AUGMENTATION)
# Modelin "pabuç" ve "ayakkabı"yı öğrenmesi için onları aynı cümlelerde geçireceğiz
genis_veri = []
for _ in range(2000): # 2000 döngü x 8 ürün = 16.000 satır
    for urun in temel_urunler:
        kelimeler = urun.split()
        random.shuffle(kelimeler) # Kelime sırasını karıştır (Model ezberlemesin)
        genis_veri.append(kelimeler)

# 3. WORD2VEC EĞİTİMİ (Parametreleri Güçlendirdik)
# vector_size: 200 (Daha fazla detay)
# window: 5 (Kelimenin sağındaki ve solundaki 5 kelimeye bak)
# sg: 1 (Skip-gram: Nadir kelimeleri yakalamada çok iyidir)
print("🧠 Model 16.000 satır veri ile eğitiliyor...")
w2v_model = Word2Vec(sentences=genis_veri, vector_size=200, window=10, min_count=1, sg=1, epochs=20)

# 4. VEKTÖRLEŞTİRME SİSTEMİ
def cumle_vektoru(metin):
    # Zeyrek ile kök bulma
    analyzer = MorphAnalyzer()
    analiz = analyzer.lemmatize(metin.lower())
    temiz_kelimeler = [item[1][0] for item in analiz]

    vektorler = [w2v_model.wv[k] for k in temiz_kelimeler if k in w2v_model.wv]
    return np.mean(vektorler, axis=0) if vektorler else np.zeros(200)

# Tüm envanteri vektöre çevir (Simülasyon için temel ürünleri kullanıyoruz)
envanter_vektorleri = np.array([cumle_vektoru(u) for u in temel_urunler])

# 5. TEST: ŞİMDİ "PABUÇ"U ANLAYACAK MI?
def ara(sorgu):
    s_vec = cumle_vektoru(sorgu).reshape(1, -1)
    benzerlikler = cosine_similarity(s_vec, envanter_vektorleri)[0]
    en_iyi = np.argmax(benzerlikler)
    skor = benzerlikler[en_iyi]

    print(f"🔎 Sorgu: {sorgu}")
    if skor > 0.1: # Bir nebze benzerlik varsa
        print(f"✨ Sonuç: {temel_urunler[en_iyi]} (Benzerlik: %{skor*100:.1f})\n")
    else:
        print("❌ Üzgünüm, ne demek istediğini anlayamadım.\n")

ara("ayağım için konforlu bir pabuç")
ara("espresso içmek istiyorum")
ara("film izlemek için büyük ekran")

🧠 Model 16.000 satır veri ile eğitiliyor...


🔎 Sorgu: ayağım için konforlu bir pabuç
✨ Sonuç: koşu ayakkabı pabuç spor konfor (Benzerlik: %93.7)



🔎 Sorgu: espresso içmek istiyorum
✨ Sonuç: kahve makine espresso çekirdek içecek (Benzerlik: %85.2)



🔎 Sorgu: film izlemek için büyük ekran
✨ Sonuç: televizyon ekran görüntü sinema panel (Benzerlik: %85.1)

